## TensorFlow CNN结构与LeNet-5、Inception-v3模型

选择环境：Anaconda Python 3.5.2  
安装Tensorflow：Python 3.5环境下运行pip install --upgrade --ignore-installed tensorflow  
参考书籍：《TensorFlow实战Google深度学习框架（第2版）》  
ipynb格式：点击阅读原文github

### 6.1 图像识别问题简介及经典数据集

●Cifar 数据集  
分为 Cifar-10 和 Cifar-100 两个问题，他们都是图像词典项目（Visual Dictionary）中 800 万张图片的一个子集。 Cifar数据集中的图片为 32×32 的彩色图片。Cifar-10 问题收集了 10 个不同种类的 60000 张图片，类别分别有：飞机，汽车，鸟，猫，鹿，狗，青蛙，马，船，卡车。    
●ImageNet 数据库  
ImageNet 是一个基于 WordNet 的大型图像数据库。在 ImageNet 中，将近 1500 万图片被关联到了 WordNet 的大约 20000 个名词同义词集上。目前每一个与 ImageNet 相关的 WordNet 同义词集都代表了现实世界的一个实体，可以被认为是分类问题中的一个类别。 ImageNet 中的图片都是从互联网上爬取下来的，并且通过亚马逊的人工标注服务将图片分类到 WordNet的同义词集上。在 ImageNet 的图片中，一张图片中可能出现多个同义词集所代表的实体。  
●ILSVRC2012 图像分类数据集  
ILSVRC2012 图像分类数据集的任务和 Cifar 数据集基本一致，也是识别图像中的主要物体。ILSVRC2012 图像分类数据集包含了来自 1000 个类别的 120 万张图片，其中每张图片属于且只属于一个类别，因为图片是直接从网上爬取得到，图片大小从几千字节到几百万字节不等。

### 6.2 卷积神经网络简介

●全连接神经网络中，每相邻两层之间的节点都有边相连，于是一般会将每一层全连接层中的节点组织成一列，这样方便显示连接结构。网络参数过多除了导致计算速度减慢，还很容易导致过拟合问题。所以需要一个更合理的神经网络来有效地减少神经网络中参数个数。 CNN 就可以达到这个目的。  
●在 CNN 中，相邻两层之间只有部分节点相连，为了展示每一层神经元的维度，一般会将每一层卷积层的节点组成一个三维矩阵。CNN 和全连接神经网络的唯一区别就在于神经网络中相邻两层的连接方式。

一个 CNN 主要由以下5种结构构成：  
1. 输入层  
三维矩阵的深度代表了图像的色彩通道。比如黑白图片的深度为1，而在RGB色彩模式下，图像的深度为3. 
2. 卷积层  
卷积层中每一个节点的输入只是上一层神经网络的一小块，这个小块常用的大小有3×3或者5×5。卷积层试图将神经网络中的每一小块进行更加深入地分析从而得到抽象程度更高的特征。一般来说，通过卷积层处理过的节点矩阵会变得更深。 
3. 池化层（Pooling）  
池化层神经网络不会改变三维矩阵的深度，池化操作可以认为是将一张分辨率较高的图片转化为分辨率较低的图片。
4. 全连接层  
在经过多轮卷积层和池化层的处理之后，在卷积神经网络的最后一般会是由1到2个全连接层来给出最后的分类结果。我们可以将卷积层和池化层看成自动图像特征提取的过程，之后仍然需要使用全连接层来完成分类任务。 
5. Softmax层  
Softmax层主要用于分类问题，得到当前样例属于不同种类的概率分布情况。 

### 6.3 卷积神经网络常用结构

#### 卷积层  
过滤器可以将当前层神经网络上的一个子节点矩阵转化为下一层神经网络上的一个单位节点矩阵。单位节点矩阵指的是一个长和宽都为1，但深度不限的节点矩阵。  
●过滤器的尺寸：一个过滤器输入节点矩阵的大小  
●过滤器的深度：输出单位节点矩阵的深度。  
为了避免矩阵尺寸的变化，可以在当前层矩阵的边界上加入全0填充（zero-padding）；此外，可以通过设置过滤器移动的步长来调整结果矩阵的大小。  
在卷积神经网络中，每一个卷积层中使用的过滤器中的参数都是一样的，这是卷积神经网络一个非常重要的性质。卷积层的参数个数和图片的大小无关，它只和过滤器的尺寸、深度以及当前层节点矩阵的深度有关，这使得卷积神经网络可以很好的扩展到更大的图像数据上。 

以下程序实现一个卷积层的前向传播过程：

In [ ]:
import tensorflow as tf
# 通过tf.get_variable的方式创建过滤器的权重变量和偏置项变量。上面介绍了卷积层
# 的参数个数只和过滤器的尺寸、深度以及当前层节点矩阵的深度有关，所以这里声明的参数变
# 量是一个四维矩阵，前面两个维度代表了过滤器的尺寸，第三个维度表示当前层的深度，第四
# 个维度表示过滤器的深度。
filter_weight = tf.get_variable(
    'weights', [5, 5, 3, 16], 
    initializer=tf.truncated_normal_initializer(stddev=0.1))
# 和卷积层的权重类似，当前层矩阵上不同位置的偏置项也是共享的，所以总共有下一层深度个
# 不同的偏置项。16为过滤器的深度，也是下一层节点矩阵的深度
biases = tf.get_variable('biases', [16], initializer=tf.constant_initializer(0.1))

# tf.nn.conv2d提供了一个非常方便的函数来实现卷积层前向传播的算法。这个函数的第一个输
# 入为当前层的节点矩阵。注意这个矩阵是一个四维矩阵，后面三个维度对应一个节点矩阵，第一
# 维对应一个输入batch。比如在输入层，input[0,:,:,:]表示第一张图片，input[1,:,:,:]
# 表示第二张图片，以此类推。tf.nn.conv2d第二个参数提供了卷积层的权重，第三个参数为不
# 同维度上的步长。虽然第三个参数提供的是一个长度为4的数组，但是第一维和最后一维的数字
# 要求一定是1。这是因为卷积层的步长只对矩阵的长和宽有效。最后一个参数是填充（padding）
# 的方法，TensorFlow提供了SAME或VALID两种选择，其中SAME表示添加全0填充，VALID表示不添加
conv = tf.nn.conv2d(input, filter_weight, strides=[1, 1, 1, 1], padding='SAME')

# tf.nn.bias_add提供了一个方便的函数给每一个节点加上偏置项。注意这里不能直接使用加
# 法，因为矩阵上不同位置上的节点都需要加上同样的偏置项。虽然下一层神经网络的大小为
# 2×2，但是偏置项只有一个数（因为深度为1），而2×2矩阵中的每一个值都需要加上这个偏置项
bias = tf.nn.bias_add(conv, biases)
# 将计算结果通过ReLU激活函数完成去线性化
actived_conv = tf.nn.relu(bias)

#### 池化层  
●使用池化层既可以加快计算速度也有防止过拟合问题的作用。  
●使用最大值操作的池化层被称之为最大池化层max pooling，这是被使用得最多的池化层结构；使用平均值操作的池化层被称之为平均池化层average pooling。   
●池化层的过滤器也需要人工设定过滤器的尺寸、是否使用全0填充以及过滤器移动的步长等设置。唯一的区别在于卷积层使用的过滤器是横跨整个深度的，而池化层使用的过滤器只影响一个深度上的节点。所以池化层的过滤器除了在长和宽两个维度移动之外，它还需要在深度这个维度移动。  
●池化层主要用于减小矩阵的长和宽。虽然池化层也可以减小矩阵深度，但是实践中一般不会这样使用。

在tfnn.max_pool函数中，首先需要传入当前层的节点矩阵，这个矩阵是一个四维矩阵，格式和tf.nn.conv2d函数中的第一个参数一致。第二个参数为过滤器尺寸。虽然给出的是一个长度为4的一维数组，但这个数组的第一个和最后一个数必须是1，这意味着池化层的过滤器是不可以跨不同输入样例或者节点矩阵深度的。在实际应用中使用最多的池化层过滤器尺寸为[1,2,2,1]或者[1,3,3,1]。  
tf.nn.avg_pool实现平均池化层，调用格式和tf.nn.max_pool函数是一致的。

以下程序实现最大池化层的前向传播算法：

In [ ]:
# tf.nn.max_pool实现了最大池化层的前向传播过程，它的参数和tf.nn.conv2d函数类似。
# ksize提供了过滤器的尺寸，strides提供了步长信息，padding提供了是否使用全0填充。
pool = tf.nn.max_pool(actived_conv, ksize=[1, 3, 3, 1], 
                      strides=[1, 2, 2, 1], padding='SAME')

#### 卷积层、池化层样例

In [2]:
# 输入矩阵
import tensorflow as tf
import numpy as np

M = np.array([
        [[1],[-1],[0]],
        [[-1],[2],[1]],
        [[0],[2],[-2]]
    ])

print("Matrix shape is: ",M.shape)

Matrix shape is:  (3, 3, 1)


In [4]:
# 定义卷积过滤器, 深度为1
tf.reset_default_graph() # 这玩意儿很重要！
filter_weight = tf.get_variable('weights', shape=[2, 2, 1, 1], 
                                initializer = tf.constant_initializer([
                                                                        [1, -1],
                                                                        [0, 2]]))
biases = tf.get_variable('biases', shape=[1], initializer = tf.constant_initializer(1))

In [5]:
# 调整输入的格式符合TensorFlow的要求
M = np.asarray(M, dtype='float32')
M = M.reshape(1, 3, 3, 1)
M

array([[[[ 1.],
         [-1.],
         [ 0.]],

        [[-1.],
         [ 2.],
         [ 1.]],

        [[ 0.],
         [ 2.],
         [-2.]]]], dtype=float32)

In [6]:
# 计算矩阵通过卷积层过滤器和池化层过滤器计算后的结果
x = tf.placeholder('float32', [1, None, None, 1])
conv = tf.nn.conv2d(x, filter_weight, strides = [1, 2, 2, 1], padding = 'SAME')
bias = tf.nn.bias_add(conv, biases)
pool = tf.nn.avg_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    convoluted_M = sess.run(bias,feed_dict={x:M})
    pooled_M = sess.run(pool,feed_dict={x:M})
    
    print("convoluted_M: \n", convoluted_M)
    print("pooled_M: \n", pooled_M)

convoluted_M: 
 [[[[ 7.]
   [ 1.]]

  [[-1.]
   [-1.]]]]
pooled_M: 
 [[[[ 0.25]
   [ 0.5 ]]

  [[ 1.  ]
   [-2.  ]]]]


### 6.4 经典卷积网络模型

#### LeNet-5模型  
layer-1 卷积层  
这一层的输入就是原始的图像像素，LeNet-5模型接收的输入层大小为32×32×1.第一个卷积层过滤器尺寸为5×5,深度为6，不使用全0填充，步长为1  
layer-2 池化层  
这一层的输入为第一层的输出，是一个28×28×6的节点矩阵。过滤器大小为2×2,长和宽的步长均为2  
layer-3 卷积层  
本层的输入矩阵大小为14×14×6，使用的过滤器大小为5×5，深度为16，不使用全0填充，步长为1.  
layer-4 池化层  
本层的输入矩阵大小为10×10×16，采用的过滤器大小为2×2,步长为2.  
layer-5 全连接层    
本层的输入矩阵大小为5×5×16，在LeNet5模型的论文中将这一层称为卷积层，但因为过滤器的大小就是5×5，所以和全连接层没有区别。  
layer-6 全连接层  
本层的输入节点个数为120个，输出节点个数为84个。  
layer-7 全连接层  
本层的输入节点个数为84个，输出节点个数为10个。LeNet5模型论文中该层结构和全连接层有区别，但这里用全连接层近似的表示。

下面给出一个TF程序来实现一个类似LeNet-5模型的卷积神经网络来解决MNIST数字识别问题。  
更新后的mnist_inference.py程序，命名为LeNet5_infernece：

In [ ]:
import tensorflow as tf

# 配置神经网络的参数
INPUT_NODE = 784
OUTPUT_NODE = 10

# 图像尺寸
IMAGE_SIZE = 28
# 图像的颜色通道数，这里只有黑白一种通道
NUM_CHANNELS = 1
# 标签的数量
NUM_LABELS = 10

# 第一层卷积的深度
CONV1_DEEP = 32
# 第一层卷积的过滤器尺寸
CONV1_SIZE = 5

# 第二层卷积的深度
CONV2_DEEP = 64
# 第二层卷积的过滤器尺寸
CONV2_SIZE = 5

# 全连接层的节点个数
FC_SIZE = 512

# 定义卷积神经网络的前向传播过程
# 这里添加了一个新的参数train，用于区分训练过程和测试过程
# 这里将用到dropout方法，进一步提升模型可靠性并防止过拟合，dropout过程只在训练时使用
def inference(input_tensor, train, regularizer):
    # 第一层卷积1
    # 输入为[x-size=28, y-size=28, channel=1]的图像
    # 过滤器尺寸[x-size=5, y-size=5, channel=1, deep=32]
    # 过滤器步长=1
    # 输出为[x-size=28, y-size=28, deep=32]的矩阵
    with tf.variable_scope('layer1-conv1'):
        conv1_weights = tf.get_variable(
            name="weight", 
            shape=[CONV1_SIZE, CONV1_SIZE, NUM_CHANNELS, CONV1_DEEP],
            initializer=tf.truncated_normal_initializer(stddev=0.1)
        )
        conv1_biases = tf.get_variable(
            name="bias", 
            shape=[CONV1_DEEP], 
            initializer=tf.constant_initializer(0.0)
        )
        conv1 = tf.nn.conv2d(input_tensor, conv1_weights, strides=[1, 1, 1, 1], padding='SAME')
        relu1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_biases))

    # 第二层池化1
    # 输入为[x-size=28, y-size=28, deep=32]的矩阵
    # 过滤器尺寸[x-size=2, y-size=2]
    # 过滤器步长=2
    # 输出为[x-size=14, y-size=14, deep=32]的矩阵
    with tf.name_scope("layer2-pool1"):
        pool1 = tf.nn.max_pool(relu1, ksize = [1,2,2,1],strides=[1,2,2,1],padding="SAME")

    # 第三层卷积2
    # 输入为[x-size=14, y-size=14, deep=32]的矩阵
    # 过滤器尺寸[x-size=5, y-size=5, channel=1, deep=64]
    # 过滤器步长=1
    # 输出为[x-size=14, y-size=14, deep=64]的矩阵
    with tf.variable_scope("layer3-conv2"):
        conv2_weights = tf.get_variable(
            name="weight", 
            shape=[CONV2_SIZE, CONV2_SIZE, CONV1_DEEP, CONV2_DEEP],
            initializer=tf.truncated_normal_initializer(stddev=0.1)
        )
        conv2_biases = tf.get_variable(
            name="bias", 
            shape=[CONV2_DEEP], 
            initializer=tf.constant_initializer(0.0)
        )
        conv2 = tf.nn.conv2d(pool1, conv2_weights, strides=[1, 1, 1, 1], padding='SAME')
        relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_biases))

    # 第四层池化2
    # 输入为[x-size=14, y-size=14, deep=64]的矩阵
    # 过滤器尺寸[x-size=2, y-size=2]
    # 过滤器步长=2
    # 输出为[x-size=7, y-size=7, deep=64]的矩阵
    with tf.name_scope("layer4-pool2"):
        pool2 = tf.nn.max_pool(relu2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        # 把[batch, x-size, y-size, deep]4维矩阵转化为[batch, vector]2维矩阵，长*宽*深度转换为1维向量
        pool_shape = pool2.get_shape().as_list()
        nodes = pool_shape[1] * pool_shape[2] * pool_shape[3]
        reshaped = tf.reshape(pool2, [pool_shape[0], nodes])

    # 全连接层    
    with tf.variable_scope('layer5-fc1'):
        fc1_weights = tf.get_variable(
            name="weight", 
            shape=[nodes, FC_SIZE],
            initializer=tf.truncated_normal_initializer(stddev=0.1)
        )
        # 只有全连接的权重需要加入正则化
        if regularizer != None: tf.add_to_collection('losses', regularizer(fc1_weights))
        fc1_biases = tf.get_variable("bias", [FC_SIZE], initializer=tf.constant_initializer(0.1))

        fc1 = tf.nn.relu(tf.matmul(reshaped, fc1_weights) + fc1_biases)
        # dropout在训练数据的时候，会随机把部分输出改为0
        # dropout可以避免过度拟合，dropout一般只在全连接层，而不是在卷积层或者池化层使用
        if train: fc1 = tf.nn.dropout(fc1, 0.5)

    # 全连接层
    # 输入为[512]的向量
    # 输出为[10]的向量
    with tf.variable_scope('layer6-fc2'):
        fc2_weights = tf.get_variable(
            name="weight", 
            shape=[FC_SIZE, NUM_LABELS],
            initializer=tf.truncated_normal_initializer(stddev=0.1)
        )
        if regularizer != None: tf.add_to_collection('losses', regularizer(fc2_weights))
        fc2_biases = tf.get_variable("bias", [NUM_LABELS], initializer=tf.constant_initializer(0.1))
        logit = tf.matmul(fc1, fc2_weights) + fc2_biases

    return logit

下面是修改后的mnist_train.py，命名为LeNet5_train，和修改前的唯一区别在于因为卷积神经网络的输入层为一个三维矩阵，所以需要调整一下输入数据的格式。

In [11]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import LeNet5_inference
import os
import numpy as np

tf.reset_default_graph() # 这玩意儿很重要！

# 定义神经网络相关的参数
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.01
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 5000
MOVING_AVERAGE_DECAY = 0.99

# 定义训练过程
def train(mnist):
    # 这里是要修改的部分
    # 定义输出为4维矩阵的placeholder
    x = tf.placeholder(tf.float32, [
            BATCH_SIZE,
            LeNet5_inference.IMAGE_SIZE,
            LeNet5_inference.IMAGE_SIZE,
            LeNet5_inference.NUM_CHANNELS],
        name='x-input')
    y_ = tf.placeholder(tf.float32, [None, LeNet5_inference.OUTPUT_NODE], name='y-input')
    
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    y = LeNet5_inference.inference(x,False,regularizer)
    global_step = tf.Variable(0, trainable=False)

    # 定义损失函数、学习率、滑动平均操作以及训练过程。
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE, LEARNING_RATE_DECAY,
        staircase=True)

    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')
        
    # 初始化TensorFlow持久化类。
    saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for i in range(TRAINING_STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            
            # 这里也是要修改的部分
            reshaped_xs = np.reshape(xs, (
                BATCH_SIZE,
                LeNet5_inference.IMAGE_SIZE,
                LeNet5_inference.IMAGE_SIZE,
                LeNet5_inference.NUM_CHANNELS))
            _, loss_value, step = sess.run(
                [train_op, loss, global_step], feed_dict={x: reshaped_xs, y_: ys})

            if i % 500 == 0:
                print("After %d training step(s), loss on training batch is %g." % (step, loss_value))

# 主程序入口
def main(argv=None):
    mnist = input_data.read_data_sets("../../datasets/MNIST_data", one_hot=True)
    train(mnist)

if __name__ == '__main__':
    main()

Extracting ../../datasets/MNIST_data\train-images-idx3-ubyte.gz
Extracting ../../datasets/MNIST_data\train-labels-idx1-ubyte.gz
Extracting ../../datasets/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../../datasets/MNIST_data\t10k-labels-idx1-ubyte.gz
After 1 training step(s), loss on training batch is 4.3806.
After 501 training step(s), loss on training batch is 0.75945.
After 1001 training step(s), loss on training batch is 0.841146.
After 1501 training step(s), loss on training batch is 0.667859.
After 2001 training step(s), loss on training batch is 0.663544.
After 2501 training step(s), loss on training batch is 0.678262.
After 3001 training step(s), loss on training batch is 0.72948.
After 3501 training step(s), loss on training batch is 0.659118.
After 4001 training step(s), loss on training batch is 0.679091.
After 4501 training step(s), loss on training batch is 0.631523.


类似地修改mnist_eval,py程序输入部分，就可以测试这个CNN在MNIST数据集上的正确率了（从单隐层神经网络正确率98.4％上升到大约99.4％）。

●一种卷积神经网络架构不能解决所有问题，比如LeNet5模型就无法很好的处理类似ImageNet这样比较大的图像数据集，下面的正则表达式公式总结了一些经典的用于图片分类问题的卷积神经网络架构： 

    输入层 ->（卷积层+->池化层？）+ -> 全连接层+ 

“卷积层+”表示一层或者多层卷积层，大部分卷积神经网络中一般最多连续使用三层卷积层。“池化层？”表示没有或者一层池化层。池化层虽然可以起到减少参数防止过拟合问题，但是在部分论文中也发现可以直接通过调整卷积层步长来完成，所以有些卷积神经网络中没有池化层。  
●一般卷积层的过滤器边长不超过5，但有些卷积神经网络中，处理输入的卷积层中使用了边长为7甚至11的过滤器。  
●在过滤器深度上，大部分卷积神经网络都采用逐层递增的方式。  
●卷积层的步长一般为1，但有些模型中也会使用 2 或者3 作为步长。池化层的配置相对简单，用得最多的是最大池化层，池化层的过滤器边长一般为 2 或者 3，步长也一般为 2 或者 3 。

#### Inception-v3模型
Inception 模块会首先使用不同尺寸的过滤器处理输入矩阵。最上方矩阵使用了边长为 1 的过滤器，中间矩阵使用的过滤器边长为 3 ，下方矩阵使用的边长为 5 。如果所有的过滤器都使用全 0 填充并且步长为 1 ，那么前向传播得到的结果矩阵的长和宽都与输入矩阵一致，经过不同过滤器处理的结果矩阵可以拼接成一个更深的矩阵。  
Inception-v3模型总共有 46 层，由 11 个 Inception 模块组成。在 Inception-v3 模型中有 96 个卷积层，如果用 LeNet-5 中卷积层实现的代码，那么一个卷积层就需要 5 行代码，96个就需要写 480 行代码来实现。代码的可读性非常差。  
为了更好的实现类似 Inception-v3 模型这样的复杂卷积神经网络， TensorFlow-Slim 工具可以更加简洁的实现一个卷积层：

In [ ]:
# 直接使用Tensorflow原始API实现卷积层
with tf.variable_scope(scope_name):
    weights = tf.get_variable('weight', ...)
    biases = tf.get_variable('bias', ...)
    conv = tf.nn.conv2d(...)
    relu = tf.nn.relu(tf.nn.bias_add(conv, biases))

# 使用TensorFlow-Slim实现卷积层，可以在一行中实现一个卷积层的前向传播算法
# slim.conv2d函数有3个参数必填，第一个是输入节点矩阵，第二个是当前卷积层过滤器的深度，第三个是
# 过滤器的尺寸。可选参数有过滤器移动的步长、是否全0填充、激活函数选择、变量命名空间等
net = slim.conv2d(input, 32, [3, 3])

完整Inception-v3模型较长，下面实现结构相对复杂的第 11 个Inception模块：

In [ ]:
# 加载slim库
slim = tf.contrib.slim

# slim.arg_scope 函数可以用于设置默认的参数取值。 slim.arg_scope 函数的第一个参数是一个函数列表，
# 这个函数列表中的函数将使用默认的参数取值，比如通过下面的定义，调用 slim.conv2d(net, 320, [1, 1])
# 函数会自动加上 stride = 1 和 padding = 'SAME' 的参数。如果在函数调用时指定了 stride ，那么这里
# 设置的默认值就不会再使用。通过这种方式可以进一步减少冗余代码。
with slim.arg_scope([sliml.con2d, slim.max_pool2d, slim.avg_pool2d], 
                    stride=1 ,padding='SAME'):
    ...
    # 此处省略了 Inception-v3 模型中其他的网络结构而直接实现最后面框中的 Inception 结构。
    # 假设输入图片经过的神经网络前向传播的结果而保存在变量 net 中。
    net = 上一次的输入节点矩阵
    # 为一个 Inception 模块声明一个统一的变量命名空间
    with tf.variable_scope('Mixed_7c'):
        # 给 Inception 模块中每一条路径声明一个命名空间
        with tf.variable_scope('Branch_0'):
            # 实现一个过滤器边长为 1，深度为 320 的卷积层。
            branch_0 = slim.conv2d(net, 320, [1, 1], scope='Conv2d_0a_1x1')
        
        # Inception 模块中第二条路径。这条计算路径上的结构本身也是一个 Inception 结构。
        with tf.variable_scope('Branch_1'):
            branch_1 = slim.conv2d(net, 384, [1, 1], scope='Conv2d_0a_1x1')
            # tf.concat 函数可以将多个矩阵拼接在一起。tf.concat 函数的第一个参数指定了拼接的维度，
            # 这里给出的"3"代表了矩阵在深度这个维度上进行拼接。
            branch_1 = tf.concat(3, [
                slim.conv2d(branch_1, 384, [1, 3], scope='Conv2d_0a_1x3'),
                slim.conv2d(branch_1, 384, [3, 1], scope='Conv2d_0a_3x1')])
        
        # Inception 模块中第三条路径。此计算路径也是一个 Inception 结构。
        with tf.variable_scope('Branch_2'):
            branch_2 = slim.conv2d(
                net, 448, [1, 1], scope='Conv2d_0a_1x1')
            branch_2 = slim.conv2d(
                branch_2, 384, [1, 3], scope='Conv2d_0a_1x3')
            branch_2 = tf.concat(3, [
                slim.conv2d(branch_2, 384, 
                            [1, 3], scope='Conv2d_0a_1x3'),
                slim.conv2d(branch_2, 384, 
                            [3, 1], scope='Conv2d_0a_3x1')])
        
        # Inception 模块中第四条路径。
        with tf.variable_scope('Branch_3'):
            branch_3 = slim.avg_pool2d(
                net, [3, 3], scope='AvgPool_0a_3x3')
            branch_3 = slim.conv2d(branch_3, 192, [1, 1], scope='Conv2d_0a_1x1')
        
        # 当前 Inception 模块的最后输出是由上面四个计算结果拼接得到的。
        # 这里的 3 表示在第三维度上进行连接。
        net = tf.concat(3, [branch_0, branch_1, branch_2, branch_3])

Github代码库上各种 Inception 模型及其它模型完整代码参考网址：  
https://github.com/tensorflow/models/tree/master/research/slim/nets  
其中 Inception-v3 以 579 行代码实现。